In [1]:
import sys, os
sys.path.insert(0, '..')

import numpy as np
import scipy.sparse as sp
import torch
import networkx as nx
import pickle as pkl
from utils.utils import sparse_to_torch_tensor, normalize_row_wise
from utils import score


In [2]:

t = 'fairwalk'
embs = pkl.load(open('/tmp/embs.pkl', 'rb'))
rgraphs = pkl.load(open("/tmp/rgraphs.pkl", 'rb'))
labels = torch.LongTensor(pkl.load(open("/tmp/group_ids.pkl", 'rb')))
adj = nx.to_scipy_sparse_matrix(rgraphs[t])
adj = sparse_to_torch_tensor(normalize_row_wise(adj + sp.eye(adj.shape[0])))
features = torch.FloatTensor(embs[t])
idx_train, idx_val = torch.LongTensor(range(80)), torch.LongTensor(range(80, 105))
features.shape, type(features), labels.shape, type(labels), adj.shape, type(adj)

(torch.Size([105, 128]),
 torch.Tensor,
 torch.Size([105]),
 torch.Tensor,
 torch.Size([105, 105]),
 torch.Tensor)

In [3]:
import math
import torch

from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module
from utils.model_utils import GraphConvolution
from utils.node_classification import GCN


In [4]:
import torch.nn as nn
import torch.nn.functional as F

In [5]:
import time
import argparse
import numpy as np

import torch
import torch.nn.functional as F
import torch.optim as optim


# Training hyperparameter configuration
class Args:
    no_cuda = False # Whether to use cuda/gpu
    seed = 42 # Set random seed
    epochs = 200 # number of iterations
    lr = 0.01 # learning rate
    weight_decay = 5e-4 # Learning rate decay
    n_hid = 16 # hidden layer dimension
    dropout = 0.5 # dropout rate

In [6]:
input_features = features.shape[1]
model = GCN(n_feat=input_features,
            n_hids=[128,32],
            n_classes=labels.max().item() + 1)

In [7]:
args = Args()
optimizer = optim.Adam(model.parameters(),
                       lr=args.lr, 
                       weight_decay=args.weight_decay)

In [8]:
y_test = model.fit(X=features, y=labels, optimizer=optimizer, adj_list=adj, log=True).transform(features, adj)

epoch:  0 loss_train: 1.0168009996414185
epoch:  1 loss_train: 0.7840081453323364
epoch:  2 loss_train: 0.6263736486434937
epoch:  3 loss_train: 0.46968892216682434
epoch:  4 loss_train: 0.44425979256629944
epoch:  5 loss_train: 0.4676939845085144
epoch:  6 loss_train: 0.44421085715293884
epoch:  7 loss_train: 0.3470110595226288
epoch:  8 loss_train: 0.36984488368034363
epoch:  9 loss_train: 0.3088047206401825
epoch:  10 loss_train: 0.3288731575012207
epoch:  11 loss_train: 0.31389638781547546
epoch:  12 loss_train: 0.3135485351085663
epoch:  13 loss_train: 0.3267526924610138
epoch:  14 loss_train: 0.3637338876724243
epoch:  15 loss_train: 0.2902345657348633
epoch:  16 loss_train: 0.29765862226486206
epoch:  17 loss_train: 0.28693172335624695
epoch:  18 loss_train: 0.28634124994277954
epoch:  19 loss_train: 0.2779911160469055
epoch:  20 loss_train: 0.30668359994888306
epoch:  21 loss_train: 0.2917424738407135
epoch:  22 loss_train: 0.28367194533348083
epoch:  23 loss_train: 0.244374811

In [9]:
# Model test function
def test():
    # First mark the model as eval mode
    model.eval()
    """
    # Input the graph network input feature and the transformed adjacency matrix adj into the two-layer graph convolutional neural network GCN model, 
    and the output is obtained through forward propagation, which is the predicted probability of the classification category
    """
    output = model(features, adj)
    """
    Find the corresponding output probability and label according to the data index of the test set, 
    and then calculate the loss and accuracy
    """
    loss_test = F.nll_loss(output[idx_val], labels[idx_val])
    acc_test = score.accuracy(output[idx_val], labels[idx_val])
    
    print("Test set results:",
          "loss= {:.4f}".format(loss_test.item()),
          "accuracy= {:.4f}".format(acc_test.item()))
    
test()

Test set results: loss= 0.1468 accuracy= 0.9200


In [10]:
features.shape

torch.Size([105, 128])

In [11]:
model(features, adj).shape

torch.Size([105, 3])

In [12]:
labels.shape

torch.Size([105])

In [13]:
input_features

128